In [1]:
######################################### IMPORTING PACAKGES #############################
from scipy import spatial
import pandas as pd
import os
import json
import numpy as np
import string
import warnings
warnings.filterwarnings("ignore")
import sys  
import os
from dateutil.parser import parse
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# PDF text extraction
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
# Others
import requests
import string
import re
from pprint import pprint
from tqdm.notebook import tqdm
import io
import nltk
from nltk.stem.snowball import SnowballStemmer
import spacy
# spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=['ner'])
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)
import pickle #pickle模块
import tensorflow as tf
from utils import extract_pdf,extract_pages_sentences,keyword_filter,custom_standardization,removeStopWords,stemSentence

In [5]:
def predict(pdf_file,model,vectorizer):
    text = extract_pdf(pdf_file)
    pages_content, pages_sentences = extract_pages_sentences(spacy.load("en_core_web_sm"),text)
    df = keyword_filter(pages_sentences,['net-zero','net-zero','carbon neutral','commitment','target','long term','2030','2040','2045','2050',' neutrality','carbon free','carbon-free','zero emission','zero GHG emission','zero CO2 emission','SBTi','Science Based Targets initiative'])
    df = df.reset_index()
    df['sentence1'] = df['sentence'].apply(lambda x:tf.compat.as_str_any(custom_standardization(x).numpy()))
    df['sentence1'] = df['sentence1'].apply(lambda x:removeStopWords(str(x)))
    df['sentence1'] = df['sentence1'].apply(lambda x:stemSentence(str(x)))
    X = vectorizer.transform(df['sentence'])
    res = model.predict(X)
    sentences = list(df[res == 1]['sentence'])
    if len(sentences)>0:
        label = 'Yes'
    else:
        label = 'False'
    return label, sentences

In [6]:
file = 'nrg-2021-sustainability-report.pdf'
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load( f)

In [7]:
predict(file,model,vectorizer)

('Yes',
 ['We are making meaningful progress in creating a more sustainable business as we continue our path to net-zero by 2050.',
  'OUR JOURNEY OUR JOURNEY We continue to make progress on our goal of being net-zero by 2050.',
  'New target is a 50% reduction by 2025 from our current 2014 baseline and net-zero by 2050.',
  'The SLBs link attractive financing to the realization of previously announced goals to achieve a 50% reduction of absolute GHG emissions by 2025 from our current 2014 baseline and reach net-zero GHG emissions by 2050.',
  'Our goals NRG is targeting a 50% reduction in GHG emissions by 2025, from our current 2014 baseline, and net-zero emissions by 2050.',
  'CO2e emissions 50% by 2025, from the current 2014 baseline, and achieve net-zero by 2050.'])